In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from google import genai

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/agent_exec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004"
)

In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "llama-3.3-70b-versatile")

In [7]:
llm.invoke("hello")

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.009940226, 'prompt_time': 0.010520218, 'queue_time': 0.211247268, 'total_time': 0.020460444}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--30ff400a-8900-452a-81e6-e8e5bb860f99-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
loader = DirectoryLoader("data", glob="./*.txt", loader_cls=TextLoader)

In [10]:
docs = loader.load()

In [11]:
docs

[Document(metadata={'source': 'data/info.txt'}, page_content='John Wick is an American media franchise created by Derek Kolstad. It centers on a neo-noir action thriller film series featuring the eponymous character portrayed by Keanu Reeves. Wick is a legendary hitman who is reluctantly drawn back into the criminal underworld after retiring. The franchise began with the release of John Wick (2014), which was followed by three sequels: Chapter 2 (2017), Chapter 3 – Parabellum (2019), and Chapter 4 (2023).[1][2] Various spin-offs expanded the franchise: the prequel comic book series John Wick: The Book of Rules (2017–2019), the prequel television miniseries The Continental (2023), and the spinoff film Ballerina (2025),[3] all incorporating elements of alternate history.\n\nThe films have received critical acclaim, and have been considered one of the greatest action film series of all time.[4][5][6][7] Some critics and publications consider the first film,[a] as well as Chapter 4,[25][26

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 50
)

In [13]:
new_docs = text_splitter.split_documents(documents=docs)

In [14]:
doc_strings = [doc.page_content for doc in new_docs]

In [15]:
db = Chroma.from_documents(new_docs, embedding=embeddings) 
retriever = db.as_retriever(search_kwargs={"k":3})

In [17]:
query = "what is llama 4?"
docs = retriever.get_relevant_documents(query)
print(docs[0].metadata)
print(docs[0].page_content)

for doc in docs:
    print(doc)

{'source': 'data/info2.txt'}
Llama 4, released in April 2025 by Meta, represents a major leap in large language model design
page_content='Llama 4, released in April 2025 by Meta, represents a major leap in large language model design' metadata={'source': 'data/info2.txt'}
page_content='scale, Llama 4 is multimodal—supporting both text and image inputs with text/code outputs—and' metadata={'source': 'data/info2.txt'}
page_content='upon release. Strategically, Llama 4 is embedded across Meta’s ecosystem—including WhatsApp,' metadata={'source': 'data/info2.txt'}


In [18]:
def function_1(AgentState):
    
    message=AgentState["messages"]
    
    question=message[-1]
    
    complete_prompt="Your task is to provide only the brief answer based on the user query. \
        Don't include too much reasoning. Following is the user query: " + question
    
    response = llm.invoke(complete_prompt)
    
    AgentState['messages'].append(response.content)
    
    return AgentState

In [19]:
def function_2(AgentState):
    messages = AgentState['messages']
    question = messages[0] 

    template = """Answer the question based only on the following context:
    {context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return result

In [20]:
# Define a Langchain graph
from langgraph.graph import Graph
workflow4 = Graph()
workflow4.add_node("LLM", function_1)
workflow4.add_node("RAGtool", function_2)
workflow4.add_edge('LLM', 'RAGtool')
workflow4.set_entry_point("LLM")
workflow4.set_finish_point("RAGtool")
app4 = workflow4.compile()

In [22]:
from IPython.display import Image, display

try:
    display(Image(app4.get_graph().draw_mermaid_png()))
except Exception as e:
    # This requires some extra dependencies and is optional
    print(e)

Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [23]:
inputs = {"messages": ["Tell me about llama 4 model"]}

In [24]:
for output in app4.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'LLM':
---
{'messages': ['Tell me about llama 4 model', 'Llama 4 is an AI model developed by Meta, designed for natural language processing and understanding.']}

---

Output from node 'RAGtool':
---
Llama 4 is a large language model released by Meta in April 2025, representing a significant advancement in large language model design. It is multimodal, supporting both text and image inputs with text/code outputs. The model lineup includes Llama 4 Scout, which is a lightweight 17B active parameter model.

---

